<a href="https://colab.research.google.com/github/swati-git/FineTuneLLM/blob/main/Transformer_Decoder_Archirecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer Architecture

In [2]:
#Get an OPT model by Facebook from huggingFace which can be found from link:https://github.com/huggingface/transformers/blob/main/src/transformers/models/auto/modeling_auto.py#L1522

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

#A causal language modeling objective trains a model to predict the next token (word, subword, or character) in a sequence based only on the previous tokens.
#This approach differs from masked language modeling (used in models like BERT), where tokens are randomly masked and the model must predict them using context from both directions.


In [ ]:
#lINK: https://medium.com/@rakeshrajpurohit/model-quantization-with-hugging-face-transformers-and-bitsandbytes-integration-b4c9983e8996
#Quantization converts model weights from high-precision formats (like 32-bit floating point, or FP32) to lower-precision formats (like 8-bit integers).
#Reduces memory usage significantly (up to 75% reduction going from FP32 to INT8)
# Speeds up computation and inference
# Allows larger models to fit on devices with limited memory
# Can reduce power consumption (important for edge devices)

# Loads the model with 8-bit integer quantization instead of the default 32-bit floating point
# Uses the bitsandbytes library to handle the quantization process
# Maps the FP32 weights to INT8 while preserving the model's performance as much as possible
# Keeps certain critical layers (like embeddings) in higher precision

#Link to Transformer APIs #https://huggingface.co/docs/transformers/model_doc/auto





In [6]:
#To resolve the error ImportError: Using `bitsandbytes` 8-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`
#!pip install -U bitsandbytes

# from transformers import BitsAndBytesConfig
# import torch

# quantization_config = BitsAndBytesConfig(
#     load_in_8bit=True,
#     bnb_4bit_compute_dtype=torch.float16
# )

# OPT = AutoModelForCausalLM.from_pretrained("facebook/opt-1.3b",quantization_config=quantization_config )

CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [7]:

OPT = AutoModelForCausalLM.from_pretrained("facebook/opt-1.3b" )

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

**Model** **Architecture**

In [8]:
print( OPT.model )

OPTModel(
  (decoder): OPTDecoder(
    (embed_tokens): Embedding(50272, 2048, padding_idx=1)
    (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
    (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (layers): ModuleList(
      (0-23): 24 x OPTDecoderLayer(
        (self_attn): OPTSdpaAttention(
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
        )
        (activation_fn): ReLU()
        (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=2048, out_features=8192, bias=True)
        (fc2): Linear(in_features=8192, out_features=2048, bias=True)
        (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      )


In [13]:
inp = "The quick brown fox jumps over the lazy dog"
inp_tokenized = tokenizer( inp, return_tensors="pt" )
print( inp_tokenized['input_ids'].size() )
print( inp_tokenized )

torch.Size([1, 10])
{'input_ids': tensor([[    2,   133,  2119,  6219, 23602, 13855,    81,     5, 22414,  2335]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [14]:
embedded_input = OPT.model.decoder.embed_tokens(inp_tokenized['input_ids'])
print( "Layer:\t", OPT.model.decoder.embed_tokens )
print( "Size:\t", embedded_input.size() )
print( "Output:\t", embedded_input )

Layer:	 Embedding(50272, 2048, padding_idx=1)
Size:	 torch.Size([1, 10, 2048])
Output:	 tensor([[[-0.0407,  0.0519,  0.0574,  ..., -0.0263, -0.0355, -0.0260],
         [-0.0371,  0.0220, -0.0096,  ...,  0.0265, -0.0166, -0.0030],
         [-0.0455, -0.0236, -0.0121,  ...,  0.0043, -0.0166,  0.0193],
         ...,
         [ 0.0007,  0.0267,  0.0257,  ...,  0.0622,  0.0421,  0.0279],
         [-0.0126,  0.0347, -0.0352,  ..., -0.0393, -0.0396, -0.0102],
         [-0.0115,  0.0319,  0.0274,  ..., -0.0472, -0.0059,  0.0341]]],
       grad_fn=<EmbeddingBackward0>)


In [15]:
embed_pos_input = OPT.model.decoder.embed_positions(inp_tokenized['attention_mask'])
print( "Layer:\t", OPT.model.decoder.embed_positions )
print( "Size:\t", embed_pos_input.size() )
print( "Output:\t", embed_pos_input )

Layer:	 OPTLearnedPositionalEmbedding(2050, 2048)
Size:	 torch.Size([1, 10, 2048])
Output:	 tensor([[[-8.1406e-03, -2.6221e-01,  6.0768e-03,  ...,  1.7273e-02,
          -5.0621e-03, -1.6220e-02],
         [-8.0585e-05,  2.5000e-01, -1.6632e-02,  ..., -1.5419e-02,
          -1.7838e-02,  2.4948e-02],
         [-9.9411e-03, -1.4978e-01,  1.7557e-03,  ...,  3.7117e-03,
          -1.6434e-02, -9.9087e-04],
         ...,
         [ 3.6979e-04, -7.7454e-02,  1.2955e-02,  ...,  3.9330e-03,
          -1.1642e-02,  7.8506e-03],
         [-2.6779e-03, -2.2446e-02, -1.6754e-02,  ..., -1.3142e-03,
          -7.8583e-03,  2.0096e-02],
         [-8.6288e-03,  1.4233e-01, -1.9012e-02,  ..., -1.8463e-02,
          -9.8572e-03,  8.7662e-03]]], grad_fn=<EmbeddingBackward0>)


In [16]:
embed_position_input = embedded_input + embed_pos_input
hidden_states, _, _ = OPT.model.decoder.layers[0].self_attn( embed_position_input )
print( "Layer:\t", OPT.model.decoder.layers[0].self_attn )
print( "Size:\t", hidden_states.size() )
print( "Output:\t", hidden_states )

Layer:	 OPTSdpaAttention(
  (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
  (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
  (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
  (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
)
Size:	 torch.Size([1, 10, 2048])
Output:	 tensor([[[-0.0135, -0.0095,  0.0013,  ...,  0.0065, -0.0017,  0.0134],
         [-0.0130, -0.0102,  0.0022,  ...,  0.0087,  0.0004,  0.0124],
         [-0.0131, -0.0059,  0.0038,  ...,  0.0098,  0.0020,  0.0141],
         ...,
         [-0.0121, -0.0098,  0.0050,  ...,  0.0095,  0.0015,  0.0098],
         [-0.0119, -0.0101,  0.0051,  ...,  0.0094,  0.0011,  0.0091],
         [-0.0118, -0.0109,  0.0055,  ...,  0.0095,  0.0013,  0.0091]]],
       grad_fn=<ViewBackward0>)


In [10]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

Tokenizationlink: https://medium.com/@lokaregns/preparing-text-data-for-transformers-tokenization-mapping-and-padding-9fbfbce28028#:~:text=The%20return_tensors%20argument%20is%20set,None%20%3A%20Returns%20Python%20lists.

torch.Size([1, 10])
{'input_ids': tensor([[    2,   133,  2119,  6219, 23602, 13855,    81,     5, 22414,  2335]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
